In [1]:
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests #HTTP操作用
import time
from selenium.webdriver.support.ui import Select
import pandas as pd
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

/Users/yutomatsuda/Lecture/DSProg2/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# ブラウザをheadlessモード実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行。コラボの場合、必須。
options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')
wait = WebDriverWait(options,10)

In [9]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)

url = "https://race.netkeiba.com/race/shutuba.html?race_id=202306050811"
driver.get(url)
time.sleep(0.1)

for a in range(1, 21):
    try:
        # 馬名とリンクの取得
        house_name=driver.find_element(By.XPATH,f'//*[@id="tr_{a}"]/td[4]/div/div/span/a').text
        print(house_name)
        house_link_element = driver.find_element(By.XPATH, f'//*[@id="tr_{a}"]/td[4]/div/div/span/a')
        house_link = house_link_element.get_attribute("href")
        print(house_link)

    except NoSuchElementException:
        # 要素が見つからなかった場合は、スキップして次の要素へ
        continue

driver.quit()

ウインマリリン
https://db.netkeiba.com/horse/2017101431
ディープボンド
https://db.netkeiba.com/horse/2017102170
ホウオウエミーズ
https://db.netkeiba.com/horse/2017104691
ヒートオンビート
https://db.netkeiba.com/horse/2017104936
アイアンバローズ
https://db.netkeiba.com/horse/2017105477
タイトルホルダー
https://db.netkeiba.com/horse/2018103559
シャフリヤール
https://db.netkeiba.com/horse/2018105165
スルーセブンシーズ
https://db.netkeiba.com/horse/2018105269
プラダリア
https://db.netkeiba.com/horse/2019100109
ライラック
https://db.netkeiba.com/horse/2019103588
スターズオンアース
https://db.netkeiba.com/horse/2019104740
ドウデュース
https://db.netkeiba.com/horse/2019105283
ジャスティンパレス
https://db.netkeiba.com/horse/2019105346
ソールオリエンス
https://db.netkeiba.com/horse/2020102899
ハーパー
https://db.netkeiba.com/horse/2020103458
タスティエーラ
https://db.netkeiba.com/horse/2020103532


In [15]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import numpy as np
import time
import re

# WebDriverのインスタンスを一度だけ生成
driver = webdriver.Chrome()
driver.implicitly_wait(10)

# URLを開く
url = "https://race.netkeiba.com/race/shutuba.html?race_id=202306050811"
driver.get(url)
time.sleep(0.1)

# 全ての馬のデータを格納するリスト
all_data = []

# 馬のリンクを取得
houselink_list = []
for a in range(1, 21):
    try:
        # 馬名とリンクの取得
        house_name_element = driver.find_element(By.XPATH, f'//*[@id="tr_{a}"]/td[4]/div/div/span/a')
        house_name = house_name_element.text
        house_link = house_name_element.get_attribute("href")
        houselink_list.append((house_name, house_link))
        print(house_name)
    except NoSuchElementException:
        # 要素が見つからなかった場合は、スキップして次の要素へ
        continue

for house_name, house_link in houselink_list:
    # 既存のブラウザセッションを使ってリンクを開く
    driver.get(house_link)
    time.sleep(0.1)
    
    # 新しいdatum辞書を作成
    datum = {'馬名': house_name}
    winscore_list = []
    wintime_list = []
    last_list = []
    weight_change_list = []

    b = 1
    while True:
        try:
            # 勝率の取得
            winscore = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[12]').text
            if winscore and winscore.isdigit():  # 数字のみを確認
                winscore_list.append(float(winscore))

            # 着差の取得（空白や**を回避）
            wintime = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[19]').text
            if wintime and wintime.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
                wintime_list.append(float(wintime))

            # 上りの取得（空白や**を回避）
            last = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[23]').text
            if last and last.replace('.', '', 1).isdigit():  # 数字または小数点を含む文字列のみを確認
                last_list.append(float(last))

            # 体重の増減の取得（空白を回避）
            weight = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[5]/div/table/tbody/tr[{b}]/td[24]').text
            match = re.search(r'\((\+|-)?(\d+)\)', weight)
            if match:  # マッチするものがあった場合
                weight_change = int(match.group(2)) * (-1 if match.group(1) == '-' else 1)
                weight_change_list.append(weight_change)

            b += 1

        except NoSuchElementException:
            # 要素が見つからない場合はループを終了
            break

    # 各データの平均値を計算して辞書に追加
    datum['勝率平均'] = np.mean(winscore_list) if winscore_list else None
    datum['着差平均'] = np.mean(wintime_list) if wintime_list else None
    datum['上り平均'] = np.mean(last_list) if last_list else None
    datum['馬体重増減平均'] = np.mean(weight_change_list) if weight_change_list else None

    # 個々の馬のデータ辞書をリストに追加
    all_data.append(datum)

# 全てのデータを出力
for horse_data in all_data:
    print(horse_data)

# ドライバーを閉じる
driver.quit()

ウインマリリン
ディープボンド
ホウオウエミーズ
ヒートオンビート
アイアンバローズ
タイトルホルダー
シャフリヤール
スルーセブンシーズ
プラダリア
ライラック
スターズオンアース
ドウデュース
ジャスティンパレス
ソールオリエンス
ハーパー
タスティエーラ
{'馬名': 'ウインマリリン', '勝率平均': 5.476190476190476, '着差平均': 1.05, '上り平均': 36.472222222222214, '馬体重増減平均': 1.0}
{'馬名': 'ディープボンド', '勝率平均': 5.12, '着差平均': 0.6736842105263158, '上り平均': 35.82727272727272, '馬体重増減平均': 0.2727272727272727}
{'馬名': 'ホウオウエミーズ', '勝率平均': 5.379310344827586, '着差平均': 0.652, '上り平均': 35.455172413793115, '馬体重増減平均': -0.06896551724137931}
{'馬名': 'ヒートオンビート', '勝率平均': 3.1923076923076925, '着差平均': 0.5608695652173913, '上り平均': 34.98461538461538, '馬体重増減平均': 0.46153846153846156}
{'馬名': 'アイアンバローズ', '勝率平均': 5.555555555555555, '着差平均': 0.8958333333333334, '上り平均': 35.51111111111111, '馬体重増減平均': 1.1111111111111112}
{'馬名': 'タイトルホルダー', '勝率平均': 3.8823529411764706, '着差平均': 0.7555555555555555, '上り平均': 35.8625, '馬体重増減平均': 0.23529411764705882}
{'馬名': 'シャフリヤール', '勝率平均': 3.3846153846153846, '着差平均': 0.6222222222222222, '上り平均': 34.75555555555556, '馬体重増減平均': -1.1111111111111112}
{'馬